In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import *
import matplotlib.pyplot as plt

In [2]:

def gyh(data):
    s=(data-data.min())/(data.max()-data.min())
    return s

def ycl(path,label):
    data=np.loadtxt(path)
    x_train=[]
    x_test=[]
    
    for i in range(10):
        data_new=data[0+i*100:102400+i*100]
        data_new=data_new.reshape(100,1024)
        np.random.shuffle(data_new)
        train=gyh(data_new[:70,:])
        test=gyh(data_new[70:,:])
        x_train.append(train)
        x_test.append(test)
        
    x_train=np.array(x_train).reshape(-1,1024)
    x_test=np.array(x_test).reshape(-1,1024)
    y_train=np.array([label for i in range(0,700)])
    y_test=np.array([label for i in range(0,300)])
    
    return x_train,y_train,x_test,y_test

def stackkk(a,b,c,d,e,f,g,h):
    aa=np.vstack((a,e))
    bb=np.hstack((b,f))
    cc=np.vstack((c,g))
    dd=np.hstack((d,h))
    return aa,bb,cc,dd

def dataset():
    '''
    0:正常轴承
    1：内圈受损
    2：滚珠受损
    3：外圈受损
    '''
    x_train1,y_train1,x_test1,y_test1=ycl('./12K_21_2/Normal_2_de.txt',0)
    x_train2,y_train2,x_test2,y_test2=ycl('./12K_21_2/IR021_2_de.txt',1)
    x_train3,y_train3,x_test3,y_test3=ycl('./12K_21_2/B021_2_de.txt',2) 
    x_train4,y_train4,x_test4,y_test4=ycl('./12K_21_2/OR021_2_6_de.txt',3)
    
    x_tr1,y_tr1,x_te1,y_te1=stackkk(x_train1,y_train1,x_test1,y_test1,x_train2,y_train2,x_test2,y_test2)
    x_tr2,y_tr2,x_te2,y_te2=stackkk(x_tr1,y_tr1,x_te1,y_te1,x_train3,y_train3,x_test3,y_test3)
    x_tr3,y_tr3,x_te3,y_te3=stackkk(x_tr2,y_tr2,x_te2,y_te2,x_train4,y_train4,x_test4,y_test4)
    
    return x_tr3,y_tr3,x_te3,y_te3
 



In [3]:
x_train,y_train,x_test,y_test=dataset()

In [4]:
x_train_=x_train.reshape(-1,32,32,1)
x_test_=x_test.reshape(-1,32,32,1)
y_train_=utils.to_categorical(y_train,num_classes=4)
y_test_=utils.to_categorical(y_test,num_classes=4)

In [5]:
train_db=tf.data.Dataset.from_tensor_slices((x_train_,y_train_))
shuffle_db=train_db.shuffle(1000)# 打乱数据
train_data=shuffle_db.batch(100)# 分组

test_db=tf.data.Dataset.from_tensor_slices((x_test_,y_test_))
test_data=test_db.batch(100)

In [6]:
# 取出一个示例
sample=next(iter(train_data))
print('sample:',sample[0].shape,sample[1].shape)

sample: (100, 32, 32, 1) (100, 4)


In [7]:
layers=[
    layers.Input(shape=[32,32,1]),
    layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='SAME',activation='relu'),
    layers.MaxPooling2D(pool_size=2,strides=2),
    #------>[16,16,64]
    
    layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='SAME',activation='relu'),
    layers.MaxPooling2D(pool_size=2,strides=2),
    #------->[8,8,128]
    
    layers.Conv2D(filters=256,kernel_size=3,strides=1,padding='SAME',activation='relu'),
    layers.MaxPooling2D(pool_size=2,strides=2),
    #------->[4,4,256]
    
    layers.Flatten(),
    #-------->[4*4*256]
    
    layers.Dense(1024,activation='relu'),
    layers.Dense(512,activation='relu'),

    layers.Dropout(0.5),
    layers.Dense(4,activation='softmax')
    
    
]

model=keras.Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 256)         295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

In [188]:
model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.categorical_crossentropy,
    metrics=[metrics.categorical_accuracy]
)

In [189]:
history=model.fit(train_data,epochs=200,validation_data=test_data)

Epoch 1/200
28/28 [==============================] - 8s 291ms/step - loss: 1.2812 - categorical_accuracy: 0.3407 - val_loss: 1.5312 - val_categorical_accuracy: 0.2500
Epoch 2/200
28/28 [==============================] - 8s 279ms/step - loss: 1.4282 - categorical_accuracy: 0.2150 - val_loss: 1.4531 - val_categorical_accuracy: 0.2500
Epoch 3/200
28/28 [==============================] - 8s 288ms/step - loss: 1.4260 - categorical_accuracy: 0.2764 - val_loss: 1.3892 - val_categorical_accuracy: 0.2500
Epoch 4/200
28/28 [==============================] - 8s 293ms/step - loss: 1.4008 - categorical_accuracy: 0.1468 - val_loss: 1.3857 - val_categorical_accuracy: 0.2500
Epoch 5/200
28/28 [==============================] - 8s 293ms/step - loss: 1.3599 - categorical_accuracy: 0.2404 - val_loss: 1.2073 - val_categorical_accuracy: 0.2783
Epoch 6/200
28/28 [==============================] - 8s 297ms/step - loss: 1.1097 - categorical_accuracy: 0.4036 - val_loss: 0.9490 - val_categorical_accuracy: 0.433

Epoch 50/200
28/28 [==============================] - 8s 295ms/step - loss: 0.0671 - categorical_accuracy: 0.9768 - val_loss: 0.0377 - val_categorical_accuracy: 0.9900
Epoch 51/200
28/28 [==============================] - 8s 298ms/step - loss: 0.0232 - categorical_accuracy: 0.9918 - val_loss: 0.0259 - val_categorical_accuracy: 0.9917
Epoch 52/200
28/28 [==============================] - 8s 295ms/step - loss: 0.0079 - categorical_accuracy: 0.9982 - val_loss: 0.0123 - val_categorical_accuracy: 0.9967
Epoch 53/200
28/28 [==============================] - 8s 298ms/step - loss: 0.0038 - categorical_accuracy: 0.9989 - val_loss: 0.0104 - val_categorical_accuracy: 0.9975
Epoch 54/200
28/28 [==============================] - 8s 295ms/step - loss: 0.0066 - categorical_accuracy: 0.9982 - val_loss: 0.0308 - val_categorical_accuracy: 0.9875
Epoch 55/200
28/28 [==============================] - 8s 295ms/step - loss: 0.0030 - categorical_accuracy: 0.9993 - val_loss: 0.0136 - val_categorical_accuracy:

Epoch 99/200
28/28 [==============================] - 8s 295ms/step - loss: 0.0198 - categorical_accuracy: 0.9936 - val_loss: 0.0277 - val_categorical_accuracy: 0.9925
Epoch 100/200
28/28 [==============================] - 8s 293ms/step - loss: 0.0037 - categorical_accuracy: 0.9986 - val_loss: 0.0269 - val_categorical_accuracy: 0.9933
Epoch 101/200
28/28 [==============================] - 8s 293ms/step - loss: 0.0032 - categorical_accuracy: 0.9993 - val_loss: 0.0155 - val_categorical_accuracy: 0.9958
Epoch 102/200
28/28 [==============================] - 8s 295ms/step - loss: 0.0027 - categorical_accuracy: 0.9993 - val_loss: 0.0187 - val_categorical_accuracy: 0.9958
Epoch 103/200
28/28 [==============================] - 8s 295ms/step - loss: 0.0129 - categorical_accuracy: 0.9954 - val_loss: 0.0489 - val_categorical_accuracy: 0.9850
Epoch 104/200
28/28 [==============================] - 8s 295ms/step - loss: 0.0027 - categorical_accuracy: 0.9993 - val_loss: 0.0174 - val_categorical_accu

28/28 [==============================] - 8s 298ms/step - loss: 0.0057 - categorical_accuracy: 0.9986 - val_loss: 0.0194 - val_categorical_accuracy: 0.9958
Epoch 196/200
28/28 [==============================] - 8s 301ms/step - loss: 0.0013 - categorical_accuracy: 1.0000 - val_loss: 0.0299 - val_categorical_accuracy: 0.9933
Epoch 197/200
28/28 [==============================] - 8s 298ms/step - loss: 0.0049 - categorical_accuracy: 0.9979 - val_loss: 0.0231 - val_categorical_accuracy: 0.9958
Epoch 198/200
28/28 [==============================] - 8s 300ms/step - loss: 0.0143 - categorical_accuracy: 0.9954 - val_loss: 0.0419 - val_categorical_accuracy: 0.9917
Epoch 199/200
28/28 [==============================] - 8s 300ms/step - loss: 0.0053 - categorical_accuracy: 0.9975 - val_loss: 0.0147 - val_categorical_accuracy: 0.9975
Epoch 200/200
28/28 [==============================] - 8s 299ms/step - loss: 0.0012 - categorical_accuracy: 0.9996 - val_loss: 0.0181 - val_categorical_accuracy: 0.9950


In [2]:
# 保存该模型
model.save('./2.model/cnn_1.h5')



NameError: name 'model' is not defined

In [194]:
#history里有个history属性，是一个字典，
history.history.keys() # 查看字典的keys

dict_keys(['loss', 'categorical_accuracy', 'val_loss', 'val_categorical_accuracy'])

In [195]:
acc=history.history['categorical_accuracy']
loss=history.history['loss']
val_acc=history.history[ 'val_categorical_accuracy']
val_loss= history.history['val_loss']

In [198]:
%matplotlib qt
plt.figure()
plt.xlim(0,200)
plt.ylim(0,1.5)
plt.plot(loss)
plt.plot(acc)
plt.plot(val_loss)
plt.plot(val_acc)
ax=plt.gca()
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')

plt.legend(['loss', 'categorical_accuracy', 'val_loss', 'val_categorical_accuracy'],loc=(1,0.5))

In [127]:
weight=model.get_weights()

In [142]:
model.predict(x_test_)

array([[1.00000000e+00, 0.00000000e+00, 1.53021796e-17, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 9.28341813e-22, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 2.36793627e-22, 0.00000000e+00],
       ...,
       [3.40366901e-11, 7.02520629e-05, 1.41241255e-11, 9.99929786e-01],
       [5.31882767e-11, 1.26213505e-04, 2.54883475e-11, 9.99873757e-01],
       [7.97244354e-11, 4.15517134e-04, 6.23236601e-11, 9.99584496e-01]],
      dtype=float32)

In [3]:
new_model=keras.models.load_model('./2.model/cnn_1.h5')

In [10]:
new_model.predict(x_test_)

array([[1.0000000e+00, 0.0000000e+00, 7.0079493e-21, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 3.5271453e-18, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 2.3907739e-21, 0.0000000e+00],
       ...,
       [5.0087556e-18, 5.5870437e-06, 2.6998601e-16, 9.9999440e-01],
       [8.7320754e-15, 8.0477321e-06, 6.7934082e-14, 9.9999189e-01],
       [8.2561818e-15, 7.2386806e-06, 6.1087359e-14, 9.9999273e-01]],
      dtype=float32)

In [11]:
pred=new_model.predict(x_test_)
pred_=np.argmax(pred,axis=1)
pred_


array([0, 0, 0, ..., 3, 3, 3], dtype=int64)

In [12]:
tf.reduce_mean(tf.cast(tf.equal(pred_,np.argmax(y_test_,axis=1)),dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=0.99916667>

In [9]:
model2=keras.Sequential([
    keras.layers.Input(shape=[8,128]),
    keras.layers.LSTM(64,dropout=0.5),
    keras.layers.Dense(512,activation='relu'),
    keras.layers.Dense(218,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4,activation='softmax')
])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_6 (Dense)              (None, 512)               33280     
_________________________________________________________________
dense_7 (Dense)              (None, 218)               111834    
_________________________________________________________________
dropout_2 (Dropout)          (None, 218)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 876       
Total params: 195,398
Trainable params: 195,398
Non-trainable params: 0
_________________________________________________________________


In [10]:
x_train2,y_train2,x_test2,y_test2=dataset()

In [11]:
x_train_2=x_train2.reshape(-1,8,128)
x_test_2=x_test2.reshape(-1,8,128)
y_train_2=keras.utils.to_categorical(y_train2,num_classes=4)

In [12]:
model2.compile(
    optimizer=optimizers.Adam(),
    loss=losses.categorical_crossentropy,
    metrics=[metrics.categorical_accuracy]
)

In [17]:
model2.fit(x_train_2,y_train_2,epochs=500)

Epoch 1/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3139 - categorical_accuracy: 0.8721
Epoch 2/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3254 - categorical_accuracy: 0.8707
Epoch 3/500
88/88 [==============================] - 0s 5ms/step - loss: 0.3375 - categorical_accuracy: 0.8686
Epoch 4/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3284 - categorical_accuracy: 0.8786
Epoch 5/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3262 - categorical_accuracy: 0.8707
Epoch 6/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3630 - categorical_accuracy: 0.8600
Epoch 7/500
88/88 [==============================] - 0s 5ms/step - loss: 0.3290 - categorical_accuracy: 0.8775
Epoch 8/500
88/88 [==============================] - 0s 5ms/step - loss: 0.3302 - categorical_accuracy: 0.8718
Epoch 9/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3115 - categorical_accuracy: 0.8868
E

88/88 [==============================] - 0s 4ms/step - loss: 0.2982 - categorical_accuracy: 0.8796
Epoch 74/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3196 - categorical_accuracy: 0.8707
Epoch 75/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2900 - categorical_accuracy: 0.8779
Epoch 76/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3021 - categorical_accuracy: 0.8793
Epoch 77/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3290 - categorical_accuracy: 0.8693
Epoch 78/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3073 - categorical_accuracy: 0.8771
Epoch 79/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3110 - categorical_accuracy: 0.8711
Epoch 80/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3067 - categorical_accuracy: 0.8775
Epoch 81/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2935 - categorical_accuracy: 0.8854
Epoch

88/88 [==============================] - 0s 4ms/step - loss: 0.3147 - categorical_accuracy: 0.8821
Epoch 146/500
88/88 [==============================] - ETA: 0s - loss: 0.2909 - categorical_accuracy: 0.88 - 0s 4ms/step - loss: 0.2932 - categorical_accuracy: 0.8861
Epoch 147/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3232 - categorical_accuracy: 0.8743
Epoch 148/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3132 - categorical_accuracy: 0.8789
Epoch 149/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3281 - categorical_accuracy: 0.8686
Epoch 150/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2858 - categorical_accuracy: 0.8846
Epoch 151/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3062 - categorical_accuracy: 0.8836
Epoch 152/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2795 - categorical_accuracy: 0.8946
Epoch 153/500
88/88 [==============================] - 0

88/88 [==============================] - 0s 4ms/step - loss: 0.2948 - categorical_accuracy: 0.8914
Epoch 217/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3191 - categorical_accuracy: 0.8814
Epoch 218/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2674 - categorical_accuracy: 0.8989
Epoch 219/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3175 - categorical_accuracy: 0.8786
Epoch 220/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2958 - categorical_accuracy: 0.8832
Epoch 221/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2577 - categorical_accuracy: 0.9018: 0s - loss: 0.2822 - categorical_ac
Epoch 222/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3200 - categorical_accuracy: 0.8736
Epoch 223/500
88/88 [==============================] - 0s 4ms/step - loss: 0.3051 - categorical_accuracy: 0.8832
Epoch 224/500
88/88 [==============================] - 0s 4ms/step - loss:

88/88 [==============================] - 0s 4ms/step - loss: 0.2612 - categorical_accuracy: 0.8946
Epoch 287/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2652 - categorical_accuracy: 0.8989
Epoch 288/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2954 - categorical_accuracy: 0.8832
Epoch 289/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2691 - categorical_accuracy: 0.8982
Epoch 290/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2650 - categorical_accuracy: 0.8982
Epoch 291/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2761 - categorical_accuracy: 0.8929
Epoch 292/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2862 - categorical_accuracy: 0.8889
Epoch 293/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2583 - categorical_accuracy: 0.9032
Epoch 294/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2540 - categorical_accuracy: 0.90

Epoch 358/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2777 - categorical_accuracy: 0.8918
Epoch 359/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2642 - categorical_accuracy: 0.9050
Epoch 360/500
88/88 [==============================] - 0s 5ms/step - loss: 0.2477 - categorical_accuracy: 0.9075
Epoch 361/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2571 - categorical_accuracy: 0.9007
Epoch 362/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2404 - categorical_accuracy: 0.9068
Epoch 363/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2522 - categorical_accuracy: 0.9129
Epoch 364/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2360 - categorical_accuracy: 0.9082
Epoch 365/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2408 - categorical_accuracy: 0.9054
Epoch 366/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2799 - categorical_

88/88 [==============================] - 0s 4ms/step - loss: 0.2154 - categorical_accuracy: 0.9171
Epoch 430/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2160 - categorical_accuracy: 0.9171
Epoch 431/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2531 - categorical_accuracy: 0.9050
Epoch 432/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2447 - categorical_accuracy: 0.9050
Epoch 433/500
88/88 [==============================] - ETA: 0s - loss: 0.2279 - categorical_accuracy: 0.91 - 0s 4ms/step - loss: 0.2290 - categorical_accuracy: 0.9157
Epoch 434/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2072 - categorical_accuracy: 0.9225
Epoch 435/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2086 - categorical_accuracy: 0.9250
Epoch 436/500
88/88 [==============================] - 0s 4ms/step - loss: 0.2241 - categorical_accuracy: 0.9189
Epoch 437/500
88/88 [==============================] - 0

88/88 [==============================] - 0s 4ms/step - loss: 0.2155 - categorical_accuracy: 0.9236


In [ ]:
model